### Data Extraction from Conference Exit Survey Comments ###

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import json

In [4]:
import openai
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

client = openai.OpenAI()

We will retrieve the comments from a .csv file and store as a list.

In [5]:
import random

comments_df = pd.read_csv('drive/MyDrive/Colab Notebooks/GBA 478/Week 3/Extraction/Conference_Exit_Comments.csv')
comments = comments_df['Comment'].tolist()
random.sample(comments, 1)

["Nada Arnot's session on Challenges facing the news industry was a highlight of the summit, offering a comprehensive analysis of the evolving landscape for news media. As the Executive Vice President of Marketing at The Economist, Arnot shared valuable insights into the challenges and opportunities facing the news industry in the digital age. Her strategic perspectives and in-depth understanding of the media landscape provided attendees with actionable takeaways for navigating the complexities of modern journalism and marketing"]

Note the addition of response_format = {"type" : "json_object"} to the chat.completions.create() function call

In [6]:
def analyze_comment(comment):
    prompt = f"""
    {comment}
    Analyse the comment above and return a JSON array as the result.
    The comments are made by attendees of a marketing conference.
    Extract the following information:

    sentiment: Sentiment of the comment

    previous_attendee:  whether or not the comment writer has attended past Apogee conferences
    Answer True if yes, False if no.  Answer Unknown if it is not possible to determine from the comment text.

    VIP:  whether or not the comment writer had VIP access to any of the events
    Answer True if yes, False if no.  Answer Unknown if it is not possible to determine from the comment text.

    planning_to_return:  whether or not the comment writer is planning to return in future years
    Answer True if yes, False if no.  Answer Unknown if it is not possible to determine from the comment text.

    speakers: Extract any speakers that are mentioned by name and output them as a comma separated Python list.

    Format the output as JSON with the following keys:
    sentiment
    previous_attendee
    VIP
    planning_to_return
    speakers
    """

    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "system", "content": "You are a comment analysis tool."},
            {"role": "user", "content": prompt}],
        temperature = 0,
        response_format={ "type": "json_object" }
    )

    response = response.choices[0].message.content

    return json.loads(response)


In [7]:
comments[3]

'The summit was a game-changer, and the opportunity to learn from industry visionaries like Anders Bering was truly invaluable. The insights gained have left me eagerly anticipating the potential for continued growth and learning at future editions of the event.'

In [10]:
response = analyze_comment(comments[3])
print(response)



{'sentiment': 'positive', 'previous_attendee': True, 'VIP': 'unknown', 'planning_to_return': True, 'speakers': ['Anders Bering']}


In [9]:
type(response)

dict

In [ ]:
response['sentiment']

'positive'

In [ ]:
response_accum = []

for comment in comments:
    response = analyze_comment(comment)
    response_accum.append(response)

In [12]:
df = pd.DataFrame(response_accum)
df.to_csv('drive/MyDrive/Colab Notebooks/GBA 478/Week 3/Extraction/reponses_output.csv', index=False, header=True)